In [1]:
%load_ext autoreload
%autoreload 2
from openscad2 import *

In [ ]:
# hirth coupling

n=10 # number of teeths
ca=20 # cone angle
ta=90 # teeth angle, angle between 2 teeths seen 
# perpendicular to the tool path
i_r=15 # inner radius
o_r=25 # outer radius
ns=20 # number of segments in each teeth
ch=1 # chamfer
fl=1.5 # fillet on the trough
l1=[[i_r,0,0],[o_r,0,0]]
l2=axis_rot_1(l1,[0,1,0],[o_r,0,0],ca)
l2=flip(line2length(l2,l_len(l2)+o_r))
l2=line2length(l2,l_len(l2)+i_r)
cone1=[ rot(f'z{i}',l2) for i in linspace(0,360,100)]
s1=[ rot(f'z{i}',l1)  for i in linspace(0,360,n+1)[:-1]]
s1=psos(cone1,s1,[0,0,1])
p1=rot(f'z{360/n/2}' ,plane([0,1,0],[1e5,1e5]))
l3=axis_rot_1(s1[0],[0,1,0],s1[0][0],-90)
l3=axis_rot_1(l3,line_as_axis(s1[0]),s1[0][0],-ta/2)

l4=axis_rot_1(flip(s1[0]),[0,1,0],s1[0][1],90)
l4=axis_rot_1(l4,line_as_axis(s1[0]),s1[0][1],-ta/2)

pnt1=ppos(p1,s1[0][0],line_as_axis(l3),unidirection=0)
pnt2=ppos(p1,s1[0][1],line_as_axis(l4),unidirection=0)
l5=[pnt1,pnt2]
c1=translate([0,0,-1e5/2],linear_extrude(arc(i_r/2,0,360,s=100),1e5))
pnt3=ppos(c1,pnt1,line_as_axis(flip(l5)))
l5p=[pnt3,pnt2]
cone2=[ rot(f'z{i}',l5p) for i in linspace(0,360,100)]
s2=[ rot(f'z{i}',l5) for i in linspace(0,360,n+1)[:-1]]
s1=[s1,s2]
s1=cpo(l_(concatenate(a_(s1).transpose(1,0,2,3))))
a=m_points1(s1[0],ns)
b=m_points1(s1[1],ns)
s1=cpo([a,b])

c1=translate([0,0,-pnt2[2]*50] ,linear_extrude(arc(i_r,0,360,s=200),pnt2[2]*100))
c2=translate([0,0,-pnt2[2]*50] ,linear_extrude(arc(o_r,0,360,s=200),pnt2[2]*100))
v1=[line_as_axis(p) for p in s1]

a=psos_v_2(c1,[cpo(s1)[0]],v1,unidirection=0)[0]
b=psos_v_2(c2,[cpo(s1)[1]],v1,unidirection=0)[0]
s1=cpo([a,b])

h=s1[ns][1][2]
s1=cpo(s1+[s1[0]])
sol1=surf_base(s1,-10)
sol2=surf_base(s1,h+10)

cone3=cpo(surface_offset(cone2,ch))
sol3=flip(rot(f'z{360/n/2}' ,surf_base(cone3,pnt2[2]+10)))

cone4=surface_offset(cone1,ch)
h1=cone4[0][0][2]
sol4=rot(f'z{360/n/2}' ,surf_base(cpo(cone4),h1-10))

s1=cpo(s1)
l1=s1[0]
l2=i_p_p(s1[:-ns],l1,fl)
l3=i_p_p(s1[ns:],l1,-fl)
f1=convert_3lines2fillet(l3,l2,l1,s=30)

l1=s1[ns]
l2=i_p_p(s1,l1,fl)
l3=i_p_p(s1,l1,-fl)
f2=convert_3lines2fillet(l3,l2,l1,s=30)
p2=translate(s1[0][0],plane(rot(f'y{ca}',[1,0,0]),[20,20]))
l1=ip_surf(p2,cpo(s1))
l1a=line2length(l1[1:3],10)
l1b=line2length(flip(l1[-3:-1]),10)
with open('trial.scad','w+') as f:
    f.write(f'''
    include<dependencies2.scad>
    //color("blue")for(p={s1})p_line3d(p,.3,1);
    //color("magenta")for(p={[l1a,l1b]})p_line3d(p,.3,1);
    //color("cyan")for(p={cone4})p_line3d(p,.3,1);
    //color("magenta")points({[pnt1]},5);
    //%{swp_surf(cone4)}
    //%{swp_surf(cone3)}
    //%{swp_surf(p1)}
    //%{swp_surf(s1)}
    //%{swp_surf(c1)}
    //%{swp_surf(c2)}
    //%{swp_surf(p2)}
    
    difference(){{
    {swp_c(flip(sol1))}
    {swp(sol3)}
    }}
    for(i=[0:360/{n}:360])rotate([0,0,i]){swp(f1)}
    /*
    translate([0,0,.1]){{
    difference(){{
    {swp_c(flip(sol2))}
    {swp(sol4)}
    }}
    for(i=[0:360/{n}:360])rotate([0,0,i]){swp(f2)}
    }}
    */
    ''')
v1=a_(l1a[1])-a_(l1a[0])
v2=a_(l1b[1]-a_(l1b[0]))
u1=v1/norm(v1)
u2=v2/norm(v2)
r2d(arccos(u1@u2))

In [ ]:
# photo_frame_support
l1=corner_radius(pts1([[0,10],[0,-10,3],[-70,0,3],q_rot2d(30,[57.5,0,5]),q_rot2d(98,[45,0])]),10)
l1=surround(l1,2)
sec=c32(axis_rot_1(corner_radius(pts1([[-14,2,.5],[12,0,.5],[0,150,.5],[-12,0,.5]]),10),[0,0,1],[-14.5,0],8))
path=corner_radius(pts1([[-1,0],[1,0,1],[0,100,1],[-1,0]]),10)
frame=prism(l1,path)
photo_frame=prism(sec,path)
with open('trial.scad','w+') as f:
    f.write(f'''
    include<dependencies2.scad>
   // color("blue")p_line3dc({l1},.3,1);
   // color("magenta")p_line3dc({sec},.3,1);
    rotate([90,0,0]){{
    {swp(frame)}
   //color("grey") {swp(photo_frame)}
    }}
    ''')

In [ ]:
# polythene clipper

a=170
l1=corner_radius(pts1([[-7,8],[6,-2,5],[1,-6,4],[110-10,0,4.9],[0,-10,4.9],[10,0,4.9],[0,15,8],q_rot2d(a,[54,0,300]),
                      q_rot2d(a+2,[53,0,4]),q_rot2d(a+2+90,[8,0,2]),q_rot2d(180,[2.5,0])]),10)

l1=surround(l1,1.5)
path=corner_radius(pts1([[-1,0],[1,0,1],[0,15,1],[-1,0]]),10)

r1=r_arc(l1[40:50])
cp1=cp_arc(l1[40:50])
c1=circle(r1-1,cp=cp1)
l1=equidistant_pathc(l1,pitch=1)
sol1=prism(l1,path)
c2=translate([0,0,-5],linear_extrude(c1,25))
s1=o_solid([1,0,0],corner_radius(pts1([[0,0,2.4],[5,0,2.4],[0,8,2.4],[-5,0,2.4]]),10),
           8,-4,-1,3.5,[0,0,30])
s2=o_solid([1,0,0],corner_radius(pts1([[0,0,2],[10,0,2],[0,10,2],[-10,0,2]]),10),13,-4,-21,10,[0,0,-15])
s3=o_solid([1,0,0],corner_radius(pts1([[0,0,2],[10,0,2],[0,10,2],[-10,0,2]]),10),13,-4,-21,-6,[0,0,-15])

# s3=o_solid([1,0,0],corner_radius(pts1([[0,0,2],[10,0,2],[0,10,2],[-10,0,2]]),10),10,-4,-25,-6,[0,0,-10])

l2=corner_radius(pts1([[7.5,2],[0,-2,1],[-2,0,1],[-2,-6,1],[-2,6,1.5],
        [-3,0,1.5],[-2,-6,1],[-2,6,1],[-2,0,1],[0,2]]),10)

l2=q_rot2d(90,l2)
path1=c23(l1[275:356])
path2=c23(l1[405:487])

s4=translate([0,0,7.5],path_extrude_open(l2,path1))
s4o=surface_offset(s4,-1)
sol2=solid_from_2surfaces(s4,s4o)
l2=corner_radius(pts1([[7.5,-5],[0,5,1],[-2,0,1],[-2,-6,1],[-2,6,1.5],
        [-3,0,1.5],[-2,-6,1],[-2,6,1],[-2,0,1],[0,-5]]),10)
l2=q_rot2d(90+180,l2)
s4=translate([0,3,7.5],path_extrude_open(l2,path2))
s4o=surface_offset(s4,1)
# a,b,c=a_(s4).shape
# s4o=l_(a_(ppplane(a_(s4).reshape(-1,3),[0,1,0],[0,0,0])).reshape(a,b,c))
sol3=solid_from_2surfaces(s4,s4o)

with open('trial.scad','w+') as f:
    f.write(f'''
    include<dependencies2.scad>
   // color("blue")p_line3d({l1},.1,1);
   // color("magenta")p_line3d({c1},.2,1);
   // color("magenta")p_line3d({l1[275:356]},.2,1);
   // color("magenta")p_line3d({l1[405:487]},.2,1);
  
    {swp(sol2)}
    {swp(sol3)}
   difference(){{
    {swp(sol1)}
    {swp(c2)}
    {swp(s1)}
    {swp(s2)}
    {swp(s3)}
}}

    
    ''')

In [ ]:
a,b,c,t,d=sym.symbols('a b c theta d')
f=b*sym.cos(t)+a*sym.sin(t)-c
t1=float(sym.N(sym.solve(f.subs([(a,10),(b,100),(c,50)]),t)[1])*180/pi)

l1=100*cos(d2r(t1))
sec=pts([[0,0],[10,0],[0,100],[-10,0]])
sec=q_rot2d(90-t1,sec)
sec=translate_2d([l1,0],sec)

with open('trial.scad','w+') as f:
    f.write(f'''
    include<dependencies2.scad>
    color("blue")p_line3dc({sec},.3,1);
     
    ''')

sec[1],sec[3]

In [ ]:
l1=100*cos(d2r(t1))
sec=pts([[0,0],[10,0],[0,100],[-10,0]])
sec=q_rot2d(90-t1,sec)
sec=translate_2d([l1,0],sec)

with open('trial.scad','w+') as f:
    f.write(f'''
    include<dependencies2.scad>
    color("blue")p_line3dc({sec},.3,1);
     
    ''')

sec[1],sec[3]

In [ ]:
def psos_v_2(s2,s3,v1,dist=100000,unidirection=0):
    '''
    project a surface on to another without loosing the original points
    surface 's3' will be projected on surface 's2'
    'v1' are the vectors for projection. v1 are the list of vectors
    '''
    p0=a_(s3).reshape(-1,3)
    f=faces_surface(len(s2),len(s2[0]))
    v=a_(s2).reshape(-1,3)
    tri=v[f]
    p2,p3,p4=tri[:,0],tri[:,1],tri[:,2]
    
    px=[]
    for i in range(len(p0)):
        n1=a_([uv(v1[i])]*len(p2))
        v2,v3=p3-p2,p4-p2
        iim=a_([n1,-v2,-v3+.0000001]).transpose(1,0,2).transpose(0,2,1)+.000001
        im=inv(iim)
        # im.shape,p0[198].shape
        t=(im@(p2-a_(p0[i])[None,:])[:,:,None]).reshape(-1,3)
        t1,t2,t3=t[:,0],t[:,1],t[:,2]
        if unidirection==0:
            dec=(t2>=0)&(t2<=1)&(t3>=0)&(t3<=1)&((t2+t3)<=1)
        elif unidirection==1:
            dec=(t1>=0)&(t2>=0)&(t2<=1)&(t3>=0)&(t3<=1)&((t2+t3)<=1)
        # im[517],inv(a_([a_([0,0,1]),-p3[517]+p2[517],-p4[517]+p2[517]]).transpose(1,0))
        if dec.any()==1 and norm(a_(n1[0])*sorted(t1[arange(len(p2))[dec]],key=abs)[0])<=dist:
            px.append(a_(p0[i])+a_(n1[0])*sorted(t1[arange(len(p2))[dec]],key=abs)[0])
        elif dec.any()==0 or norm(a_(n1[0])*sorted(t1[arange(len(p2))[dec]],key=abs)[0])>dist:
            px.append(p0[i])
    
    px=l_(a_(px).reshape(len(s3),len(s3[0]),3))
    return px